In [1]:
import os
import platform
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import time
import random

if platform.system() == 'Darwin':  # macOS
    base_FP = '/Users/lsj'
    cpuserver_data_FP = base_FP + '/cpuserver_data'
    nas_data_FP = '/Volumes/qnap_nas'
elif platform.system() == 'Linux':  # Linux systems (Workstation / CPU Server GPU Server)
    base_FP = '/home/seongjun'
    cpuserver_data_FP = base_FP + '/cpuserver_data' # Workstation / GPU Server
    if not os.path.exists(cpuserver_data_FP):
        cpuserver_data_FP = '/data' # CPU Server
    nas_data_FP = base_FP + '/NAS'

## Define parameters

In [2]:
# Define parameters
save_dir = os.path.join(nas_data_FP, 'water_quality/WEIS_data') # Change your path
url = "http://apis.data.go.kr/1480523/WaterQualityService/getWaterMeasuringList?pageNo=1&resultType=xml"
serviceKey = "&serviceKey=xRUuN88FgTSyqrTnD4CS9dijQGMOlS8bI0m5KwmQwF36OHos33wW9BYIfHjzyFuH3aRThLrdRtE4Rx5rYKAZ2A%3D%3D"
numOfRows = "&numOfRows=1000"
wmyrList = "&wmyrList=2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024"

## Check total number of items

In [3]:
# Check total number of items
result = requests.get(url + serviceKey + numOfRows + wmyrList)
soup = BeautifulSoup(result.text,'xml')
total_count_tag = soup.find("totalCount")
if total_count_tag:
    total_count = int(total_count_tag.text)
else:
    total_count = 0
items = soup.find_all("item")
print(f'Total number of items: {total_count}')

Total number of items: 434375


## Data Collection

In [17]:
# Collect data from all pages
all_items_data = []  # List to store all data
rows_per_page = 1000 # numOfRows
total_pages = (total_count // rows_per_page) + 1 # Total number of pages

print(f"\nTotal {total_count} data will be collected in {total_pages} pages.")

# Collect data from all pages
for page in tqdm(range(1, total_pages + 1), desc="Collecting data"):
    
    # Try up to 3 times
    for attempt in range(3):
        try:
            # Create URL for each page
            url_with_pagination = f"http://apis.data.go.kr/1480523/WaterQualityService/getWaterMeasuringList?pageNo={page}&resultType=xml" + \
                                  serviceKey + \
                                  f"&numOfRows={rows_per_page}" + \
                                  wmyrList
            
            # Request data
            response = requests.get(url_with_pagination, timeout=30)
            
            # If request is successful
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'xml')
                items = soup.find_all("item")
                
                # If items are normal, extract data and stop retry
                if items:
                    for item in items:
                        row_data = {child.name: child.text for child in item.find_all()}
                        all_items_data.append(row_data)
                    break # Current page collection is successful, move to next page
            
            # If request fails or items are not found, wait and retry
            time.sleep(random.uniform(1, 3))

        except requests.exceptions.RequestException as e:
            # If network error occurs, wait and retry
            time.sleep(random.uniform(2, 5))
            
# Convert collected data to dataframe
df = pd.DataFrame(all_items_data)

# Check final result
print("--- Data collection completed ---")
print(f"Total number of data provided by API: {total_count}")
print(f"Actual number of collected data: {len(df)}")
display(df)

페이지 수집 중:   0%|          | 0/435 [00:00<?, ?it/s]


--- 데이터 수집 완료 ---
API가 제공하는 전체 데이터 수: 434375
실제로 수집된 데이터 수: 424375


,rowno,ptNo,ptNm,addr,orgNm,wmyr,wmod,wmwk,lonDgr,lonMin,...,itemChcl3,itemToc,itemDehp,itemAntimon,itemDiox,itemHcho,itemHcb,itemNi,itemBa,itemSe
0,1,5004A25,금천,전라남도 나주시 세지면 성산리 마을교량,영산강물환경연구소,2016,03,5회차,126,44,...,,3.6,,,,,,,,
1,2,5006A25,무안천,전라남도 무안군 무안읍 용월리 송촌교상류교량,영산강물환경연구소,2016,03,4회차,126,29,...,,8.0,,,,,,,,
2,3,5006A45,삼포천1,전라남도 나주시 공산면 복용리 새마을교,영산강물환경연구소,2016,03,3회차,126,37,...,,6.2,,,,,,,,
3,4,2002A36,길안천1,경상북도 청송군 안덕면 고와리 고와2교,낙동강물환경연구소,2016,03,1회차,128,58,...,,2.9,,,,,,,,
4,5,2020A10,남지,경상남도 함안군 칠서면 계내리 남지교,낙동강유역환경청,2016,03,2회차,128,28,...,0.0000,2.7,,0.0000,0.000,0.000,0.000,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424370,434371,3302D70,석우저수지,전라북도 정읍시 고부면 장문리,한국농어촌공사,2023,07,5회차,126,46,...,,6.9,,,,,,,,
424371,434372,3302D85,백산저수지,전라북도 김제시 백산면 하정리,한국농어촌공사,2023,07,5회차,126,53,...,,4.2,,,,,,,,
424372,434373,3303D40,청호저수지,전라북도 부안군 하서면 청호리,한국농어촌공사,2023,07,5회차,126,40,...,,5.6,,,,,,,,
424373,434374,4006D30,구성저수지,전라남도 곡성군 오곡면 구성리,한국농어촌공사,2023,07,5회차,127,17,...,,3.1,,,,,,,,


### (Optional) Save Raw Dataframe

In [18]:
# Save dataframe to csv
df.to_csv(os.path.join(save_dir, 'WEIS_obs_data_2013_2024.csv'), index=False, encoding='utf-8-sig')

### (Optional) Load Raw Dataframe

In [5]:
# Load csv file
file_path = os.path.join(save_dir, 'WEIS_obs_data_2013_2024.csv')
df_original = pd.read_csv(file_path, low_memory=False)

df_original.head()

,rowno,ptNo,ptNm,addr,orgNm,wmyr,wmod,wmwk,lonDgr,lonMin,...,itemChcl3,itemToc,itemDehp,itemAntimon,itemDiox,itemHcho,itemHcb,itemNi,itemBa,itemSe
0,1,5004A25,금천,전라남도 나주시 세지면 성산리 마을교량,영산강물환경연구소,2016,3,5회차,126.0,44.0,...,NaN,3.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,5006A25,무안천,전라남도 무안군 무안읍 용월리 송촌교상류교량,영산강물환경연구소,2016,3,4회차,126.0,29.0,...,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,5006A45,삼포천1,전라남도 나주시 공산면 복용리 새마을교,영산강물환경연구소,2016,3,3회차,126.0,37.0,...,NaN,6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2002A36,길안천1,경상북도 청송군 안덕면 고와리 고와2교,낙동강물환경연구소,2016,3,1회차,128.0,58.0,...,NaN,2.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2020A10,남지,경상남도 함안군 칠서면 계내리 남지교,낙동강유역환경청,2016,3,2회차,128.0,28.0,...,0.0000,2.7,NaN,0.0000,0.000,0.000,0.000,NaN,NaN,NaN


## Convert DMS to Decimal

In [7]:
# Convert DMS(Degrees, Minutes, Seconds) related columns to numeric (NaN if error occurs)
dms_cols = ['lonDgr', 'lonMin', 'lonSec', 'latDgr', 'latMin', 'latSec']
for col in dms_cols:
    df_original[col] = pd.to_numeric(df_original[col], errors='coerce')

# Fill NaN values with 0
df_original[dms_cols] = df_original[dms_cols].fillna(0)

# Calculate decimal degrees for latitude and longitude
df_original['latitude'] = df_original['latDgr'] + (df_original['latMin'] / 60) + (df_original['latSec'] / 3600)
df_original['longitude'] = df_original['lonDgr'] + (df_original['lonMin'] / 60) + (df_original['lonSec'] / 3600)

# Check the result
# Print converted latitude and longitude columns together to easily compare the results
result_cols = [
    'latDgr', 'latMin', 'latSec', 'latitude',
    'lonDgr', 'lonMin', 'lonSec', 'longitude'
]
print(df_original[result_cols].head())

   latDgr  latMin  latSec   latitude  lonDgr  lonMin  lonSec   longitude
0    34.0    54.0     3.0  34.900833   126.0    44.0    29.0  126.741389
1    35.0     0.0    14.0  35.003889   126.0    29.0    47.0  126.496389
2    34.0    55.0    13.0  34.920278   126.0    37.0    51.0  126.630833
3    36.0    22.0     7.0  36.368611   128.0    58.0    17.0  128.971389
4    35.0    22.0    51.0  35.380833   128.0    28.0    25.0  128.473611


## Remove Blank Rows

In [49]:
all_zero_rows = (df_original[dms_cols] == 0).all(axis=1)

# Filter rows that meet the condition
blank_dms_df = df_original[all_zero_rows]

# Remove rows that meet the condition
df_filtered = df_original[~all_zero_rows].copy()

# Select columns
final_cols = [
    'ptNo', 'ptNm', 'latitude', 'longitude', 'wmcymd', 
    'wmdep', 'itemTemp', 'itemCloa', 'itemTn', 'itemTp', 'itemDoc'
]

df_filtered = df_filtered[final_cols]

# Sort values
df_filtered = df_filtered.sort_values(by=['ptNo', 'wmcymd'])

# Convert data type
## 1. wmcymd
df_filtered['wmcymd'] = pd.to_datetime(df_filtered['wmcymd'], format='%Y.%m.%d', errors='coerce')

# Remove rows that have NaN in wmcymd
df_filtered.dropna(subset=['wmcymd'], inplace=True)

## 2. ptNo, ptNm
df_filtered['ptNo'] = df_filtered['ptNo'].astype(str)
df_filtered['ptNm'] = df_filtered['ptNm'].astype(str)
df_filtered['wmdep'] = df_filtered['wmdep'].astype(str)

## 3. latitude, longitude, wmdep, itemTemp, itemCloa, itemTn, itemTp, itemDoc
float_cols = [
    'latitude', 'longitude', 'wmdep', 'itemTemp', 
    'itemCloa', 'itemTn', 'itemTp', 'itemDoc'
]
for col in float_cols:
    df_filtered[col] = pd.to_numeric(df_filtered[col], errors='coerce')

# Remove rows that are all NaN
subset_cols = df_filtered.columns[5:]
df_filtered.dropna(subset=subset_cols, how='all', inplace=True)

# Remove rows that have NaN in itemCloa
df_filtered.dropna(subset=['itemCloa'], inplace=True)
df_filtered['itemCloa'] = df_filtered['itemCloa'].replace(0, 0.05) # Replace 0 with 0.00005

# Sort values
df_filtered = df_filtered.sort_values(by=['ptNo', 'wmcymd', 'wmdep'], na_position='last')

# Remove duplicate rows
df_final = df_filtered.drop_duplicates(subset=['ptNo', 'wmcymd'], keep='first')

# Reset index
df_final.reset_index(drop=True, inplace=True)

df_final

,ptNo,ptNm,latitude,longitude,wmcymd,wmdep,itemTemp,itemCloa,itemTn,itemTp,itemDoc
0,1001A05,송천1,37.658611,128.676389,2013-03-21,NaN,2.5,1.50,3.785,0.020,10.5
1,1001A05,송천1,37.658611,128.676389,2013-04-04,NaN,10.7,2.80,3.342,0.023,12.6
2,1001A05,송천1,37.658611,128.676389,2013-05-29,NaN,17.8,5.50,2.748,0.026,10.2
3,1001A05,송천1,37.658611,128.676389,2013-06-25,NaN,23.4,10.30,5.049,0.135,9.4
4,1001A05,송천1,37.658611,128.676389,2013-07-04,NaN,20.6,3.90,4.419,0.045,9.7
...,...,...,...,...,...,...,...,...,...,...,...
245297,6003A10,동홍천,33.246944,126.571944,2024-08-14,NaN,17.1,0.10,2.036,0.081,9.0
245298,6003A10,동홍천,33.246944,126.571944,2024-09-06,NaN,16.8,0.30,1.480,0.089,9.1
245299,6003A10,동홍천,33.246944,126.571944,2024-10-24,NaN,15.7,0.05,1.212,0.098,9.2
245300,6003A10,동홍천,33.246944,126.571944,2024-11-14,NaN,15.8,0.05,1.162,0.098,9.5


In [50]:
save_dir = os.path.join(nas_data_FP, 'water_quality/WEIS_data')
os.makedirs(save_dir, exist_ok=True)
file_name = "WEIS_obs_filtered_2013_2024.csv"
save_path = os.path.join(save_dir, file_name)

df_final.to_csv(save_path, encoding='utf-8-sig', index=False)